# Professional Job Application Toolkit

A streamlined toolkit for email sending and cover letter generation.

## Features
- Email sending with your existing email template
- Cover letter generation with your existing template  
- PDF export functionality
- Simple variable input system

## 1. Import Libraries and Configuration

In [32]:
# Import required libraries
import smtplib
import ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import os
from datetime import datetime
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from dotenv import load_dotenv

# For PDF conversion
try:
    from docx2pdf import convert
    PDF_CONVERSION_AVAILABLE = True
    print("✅ PDF conversion library available")
except ImportError:
    PDF_CONVERSION_AVAILABLE = False
    print("⚠️ docx2pdf not available. Install with: pip install docx2pdf")

# Load environment variables from .env file
load_dotenv()

# Gmail Configuration
GMAIL_EMAIL = os.getenv('GMAIL_EMAIL')
GMAIL_PASSWORD = os.getenv('GMAIL_PASSWORD')
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587

✅ PDF conversion library available


## 2. Template Functions (From Your LetterMaker.py)

In [33]:
def coverLetter(company_name, role, industry):
    """Generate cover letter using your existing template"""
    current_date = datetime.now().strftime("%B %d, %Y")
    output = f"""{current_date}
{company_name}

To Whom it May Concern,

I am writing to express my enthusiastic interest in the {role} position at {company_name}. As a third-year student majoring in Business Computing & Data Analytics at Hong Kong Baptist University, I have developed a robust foundation in both software development and finance, and I am eager to leverage these skills within {company_name}'s dynamic environment. Your commitment to innovation and leadership in the {industry} industry is particularly inspiring, and I am drawn to your dedication to using technology to drive financial solutions. The opportunity to work with your extensive resources and industry expertise aligns perfectly with my passion.

My recent experience as a Software Engineer Intern at Wizpresso has been pivotal in enhancing my understanding of software engineering and its application in financial technology. Over the summer, I was responsible for refactoring the user management portal, where I developed 15 new features and optimized 10 existing ones to improve portal speed and user engagement. This role underscored the critical importance of delivering fast, reliable, and intuitive software in financial services, where every millisecond can impact trading and user satisfaction. Additionally, I refactored the user management system to enhance efficiency and scalability, skills that are crucial for handling large datasets and complex transactions in a tech-driven institution like {company_name}. Working directly with sensitive financial data from various companies, I routed APIs to dynamically display data to applications addressing stringent requirements around security, reliability and responsiveness within regulated financial markets, enabling our data science team to manage information accurately.

At Admazes, I had the opportunity to contribute significantly as a Software Engineer Intern. During my tenure, I worked on developing and enhancing a Local Large Language Model using LangChain. This project allowed me to improve the model's response accuracy by 25% through better semantic understanding and advanced search capabilities. Additionally, I analyzed datasets using BigQuery, optimizing data retrieval efficiency by 30%. By crafting and executing over 100 complex SQL queries, I was able to generate actionable insights that led to a 20% boost in operational efficiency. This experience reinforced the importance of precise and efficient data processing—skills that are vital in the {industry} world. The {role} position aligns perfectly with my aspiration to merge technology with finance. The prospect of collaborating with passionate individuals from diverse backgrounds, including fellow {industry} enthusiasts and experts from {company_name}, is incredibly motivating. I am excited about the chance to exchange ideas and skills and contribute to projects that can create a lasting positive impact. I am confident that my technical skills, combined with my enthusiasm for innovation and teamwork, make me a strong candidate for the {role} position.

Thank you for considering my application. I look forward to the possibility of contributing to {company_name} and making a meaningful difference through technology.

"""
    return output

def emailLetter(company_name, role):
    """Generate email using your existing template"""
    output = f"""Dear {company_name} Team,

I am currently studying at Hong Kong Baptist University, majoring in Business Computing & Data Analytics, from which I will be graduating in May 2026. It is with great interest that I am applying for the {role} position at {company_name}.

I am confident that my technical qualifications and work experiences developing solutions for clients and end users make me an excellent candidate for this internship. I would appreciate the opportunity to discuss how I can help support {company_name}'s needs.

Thank you for your consideration.
Attached you will find my CV and recommendation letters.

Yours sincerely,
Felix Lau Pangestu
"""
    return output

print("✅ Template functions loaded successfully!")

✅ Template functions loaded successfully!


## 3. Input Variables

In [42]:
# ===== INPUT VARIABLES - MODIFY THESE FOR EACH APPLICATION =====



# Company and position details
company_name = input('Enter company name: ')
position = input('Enter position: ')
industry = input('Enter industry: ')
recipient_email = input('Enter recipient email: ')
# Email details
email_subject = f'Application for {position} Position'  # Email subject

# Attachment paths
attachment_paths = ['Felix Lau Pangestu_CV.pdf',f'cover_letters\cover_letter_{company_name}.pdf', 'Transcript_HKBU.pdf', 'Recommendation_Letters.pdf']  # List of file paths to attach

print("📝 Variables configured:")
print(f"Company: {company_name}")
print(f"Position: {position}")
print(f"Industry: {industry}")
print(f"Recipient: {recipient_email}")
print(f"Subject: {email_subject}")

📝 Variables configured:
Company: Cathay
Position: Graduate Trainee
Industry: aviation
Recipient: chokinghazarddd@gmail.com
Subject: Application for Graduate Trainee Position


## 4. Email Sending Function

In [43]:
def send_email(to_email, subject, body, attachment_path=None):
    """Send email using Gmail SMTP"""
    try:
        # Create message
        msg = MIMEMultipart()
        msg['From'] = GMAIL_EMAIL
        msg['To'] = to_email
        msg['Subject'] = subject
        
        # Add body
        msg.attach(MIMEText(body, 'plain'))
        
        # Initialize attachment path as list if not already
        if attachment_path and not isinstance(attachment_path, list):
            attachment_path = [attachment_path]
        elif not attachment_path:
            attachment_path = []

        # Add multiple attachments if provided
        for single_attachment in attachment_path:
            if os.path.exists(single_attachment):
                with open(single_attachment, "rb") as attachment:
                    part = MIMEBase('application', 'octet-stream')
                    part.set_payload(attachment.read())
                
                encoders.encode_base64(part)
                part.add_header(
                    'Content-Disposition',
                    f'attachment; filename= {os.path.basename(single_attachment)}'
                )
                msg.attach(part)
                print(f"📎 Attachment added: {os.path.basename(single_attachment)}")
            else:
                print(f"⚠️ Attachment not found: {single_attachment}")
        
        # Send email
        context = ssl.create_default_context()
        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls(context=context)
            server.login(GMAIL_EMAIL, GMAIL_PASSWORD)
            server.sendmail(GMAIL_EMAIL, to_email, msg.as_string())
        
        print(f"✅ Email sent successfully to {to_email}")
        return True
        
    except Exception as e:
        print(f"❌ Failed to send email: {str(e)}")
        return False

## 5. PDF Export Function

In [44]:
def export_cover_letter_to_word_and_pdf(content, company_name, word_filename=None, pdf_filename=None):
    """Export cover letter content to Word document and PDF with traditional formatting"""
    try:
        # Create cover_letters directory if it doesn't exist (only for PDF)
        cover_letters_dir = "cover_letters"
        if not os.path.exists(cover_letters_dir):
            os.makedirs(cover_letters_dir)
            print(f"✅ Created directory: {cover_letters_dir}")
        
        # Set default filenames if not provided
        if word_filename is None:
            word_filename = "Cover_Letter.docx"  # Save in main directory
        if pdf_filename is None:
            pdf_filename = f"cover_letter_{company_name}.pdf"
            
        # Create full paths
        word_path = word_filename  # Save Word doc in main directory
        pdf_path = os.path.join(cover_letters_dir, pdf_filename)  # Save PDF in cover_letters folder
        
        # Create a new document
        doc = Document()
        
        # Set up document margins (1 inch on all sides)
        sections = doc.sections
        for section in sections:
            section.top_margin = Inches(1)
            section.bottom_margin = Inches(1)
            section.left_margin = Inches(1)
            section.right_margin = Inches(1)
        
        # Parse the content to extract components
        lines = content.strip().split('\n')
        
        # Extract date (first line)
        date_line = lines[0] if lines else datetime.now().strftime("%B %d, %Y")
        
        # Extract company name (second line)
        company_line = lines[1] if len(lines) > 1 else company_name
        
        # Your contact information (header)
        header = doc.add_paragraph()
        header.alignment = WD_ALIGN_PARAGRAPH.RIGHT
        header.add_run("Felix Lau Pangestu\n").bold = True
        header.add_run(f"{GMAIL_EMAIL}\n")
        header.add_run("(+852) 9438 1766")  # Add phone if available
        
        # Add spacing
        doc.add_paragraph()
        
        # Date
        date_para = doc.add_paragraph(date_line)
        date_para.alignment = WD_ALIGN_PARAGRAPH.LEFT
        
        # Add spacing
        doc.add_paragraph()
        
        # Recipient information
        recipient_para = doc.add_paragraph()
        recipient_para.alignment = WD_ALIGN_PARAGRAPH.LEFT
        recipient_para.add_run(f"{company_name}\n").bold = True
        recipient_para.add_run("Hiring Manager\n")
        recipient_para.add_run(f"{company_name} Human Resources Department")
        
        # Add spacing
        doc.add_paragraph()
        
        # Find the salutation and body content
        salutation_found = False
        body_started = False
        
        for i, line in enumerate(lines):
            line = line.strip()
            
            # Skip the date and company name lines
            if i <= 1:
                continue
                
            # Add salutation
            if line.startswith("To Whom it May Concern") or line.startswith("Dear"):
                salutation_para = doc.add_paragraph(line)
                salutation_para.alignment = WD_ALIGN_PARAGRAPH.LEFT
                doc.add_paragraph()  # Add spacing after salutation
                salutation_found = True
                body_started = True
                continue
            
            # Start body content after salutation
            if salutation_found and body_started:
                if line:  # Non-empty line
                    # Check if it's a closing line
                    if any(closing in line.lower() for closing in ["thank you", "sincerely", "yours", "best regards"]):
                        # Add closing
                        doc.add_paragraph()  # Add spacing before closing
                        closing_para = doc.add_paragraph(line)
                        closing_para.alignment = WD_ALIGN_PARAGRAPH.LEFT
                        
                        # Add signature space
                        doc.add_paragraph()
                        doc.add_paragraph()
                        
                        # Add signature line
                        signature_para = doc.add_paragraph("Felix Lau Pangestu")
                        signature_para.alignment = WD_ALIGN_PARAGRAPH.LEFT
                        break
                    else:
                        # Regular body paragraph
                        body_para = doc.add_paragraph(line)
                        body_para.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
                        
                        # Add spacing between paragraphs
                        if i < len(lines) - 1 and lines[i + 1].strip():
                            doc.add_paragraph()
        
        # Save the Word document in main directory
        doc.save(word_path)
        print(f"✅ Word document created: {word_path}")
        
        # Convert to PDF and save in cover_letters folder if library is available
        if PDF_CONVERSION_AVAILABLE:
            try:
                convert(word_path, pdf_path)
                print(f"✅ PDF document created: {pdf_path}")
                return {'word_path': os.path.abspath(word_path), 'pdf_path': os.path.abspath(pdf_path)}
            except Exception as e:
                print(f"⚠️ PDF conversion failed: {str(e)}")
                return {'word_path': os.path.abspath(word_path), 'pdf_path': None}
        else:
            print("⚠️ PDF conversion skipped - docx2pdf not available")
            print("   Install with: pip install docx2pdf")
            return {'word_path': os.path.abspath(word_path), 'pdf_path': None}
        
    except Exception as e:
        print(f"❌ Failed to create documents: {str(e)}")
        return {'word_path': None, 'pdf_path': None}

# Keep the old function for backward compatibility
def export_cover_letter_to_word(content, company_name, filename):
    """Legacy function - now calls the new function"""
    result = export_cover_letter_to_word_and_pdf(content, company_name, filename)
    return result['word_path'] if result['word_path'] else None

print("📄 Word document and PDF export functions ready!")

📄 Word document and PDF export functions ready!


In [45]:
# Generate cover letter and export to Word and PDF
cover_letter_content = coverLetter(company_name, position, industry)
export_result = export_cover_letter_to_word_and_pdf(cover_letter_content, company_name)

# Store paths for later use
word_path = export_result['word_path']
pdf_path = export_result['pdf_path']

print("\n📄 FILES CREATED:")
if word_path:
    print(f"✅ Word document: {word_path}")
if pdf_path:
    print(f"✅ PDF document: {pdf_path}")
else:
    print("⚠️ PDF not created - install docx2pdf library")

✅ Word document created: Cover_Letter.docx


  0%|          | 0/1 [00:00<?, ?it/s]

✅ PDF document created: cover_letters\cover_letter_Cathay.pdf

📄 FILES CREATED:
✅ Word document: c:\Users\j_fel\Desktop\jobApplications\Cover_Letter.docx
✅ PDF document: c:\Users\j_fel\Desktop\jobApplications\cover_letters\cover_letter_Cathay.pdf


## 6. Generate Content

In [46]:
# Generate email content
email_content = emailLetter(company_name, position)

print("\n📧 EMAIL CONTENT:")
print("=" * 50)
print(email_content[:500] + "..." if len(email_content) > 500 else email_content)

print("\n📄 COVER LETTER CONTENT:")
print("=" * 50)
print(cover_letter_content[:500] + "..." if len(cover_letter_content) > 500 else cover_letter_content)


📧 EMAIL CONTENT:
Dear Cathay Team,

I am currently studying at Hong Kong Baptist University, majoring in Business Computing & Data Analytics, from which I will be graduating in May 2026. It is with great interest that I am applying for the Graduate Trainee position at Cathay.

I am confident that my technical qualifications and work experiences developing solutions for clients and end users make me an excellent candidate for this internship. I would appreciate the opportunity to discuss how I can help support Ca...

📄 COVER LETTER CONTENT:
October 02, 2025
Cathay

To Whom it May Concern,

I am writing to express my enthusiastic interest in the Graduate Trainee position at Cathay. As a third-year student majoring in Business Computing & Data Analytics at Hong Kong Baptist University, I have developed a robust foundation in both software development and finance, and I am eager to leverage these skills within Cathay's dynamic environment. Your commitment to innovation and leadership in t

## 8. Send Email (Uncomment to Execute)

In [47]:
# SEND EMAIL - Uncomment the line below to actually send the email
send_email(recipient_email, email_subject, email_content, attachment_paths)

print("⚠️  Email sending is commented out for safety.")
print("⚠️  Uncomment the send_email line above to actually send the email.")
print("\n📋 SUMMARY:")
print(f"✅ Email content generated using your template")
print(f"✅ Cover letter content generated using your template")
print(f"✅ Word document exported: {word_path if word_path else 'Failed'}")
print(f"✅ PDF document exported: {pdf_path if pdf_path else 'Failed - install docx2pdf'}")
print(f"🎯 Ready to send to: {recipient_email}")
print(f"📧 Subject: {email_subject}")
print(f"📎 Attachments: {', '.join([os.path.basename(path) for path in attachment_paths]) if attachment_paths else 'None'}")

print("\n📄 TO USE THIS TOOLKIT:")
print("1. Modify the variables in Section 3")
print("2. Run all cells")
print("3. Uncomment the send_email line above to send")
print("4. Install docx2pdf for PDF conversion: pip install docx2pdf")

print(f"\n✅ All files ready in: {os.getcwd()}")
print(f"📁 Cover letters saved in: {os.path.join(os.getcwd(), 'cover_letters')}")

📎 Attachment added: Felix Lau Pangestu_CV.pdf
📎 Attachment added: cover_letter_Cathay.pdf
📎 Attachment added: Transcript_HKBU.pdf
📎 Attachment added: Recommendation_Letters.pdf
✅ Email sent successfully to chokinghazarddd@gmail.com
⚠️  Email sending is commented out for safety.
⚠️  Uncomment the send_email line above to actually send the email.

📋 SUMMARY:
✅ Email content generated using your template
✅ Cover letter content generated using your template
✅ Word document exported: c:\Users\j_fel\Desktop\jobApplications\Cover_Letter.docx
✅ PDF document exported: c:\Users\j_fel\Desktop\jobApplications\cover_letters\cover_letter_Cathay.pdf
🎯 Ready to send to: chokinghazarddd@gmail.com
📧 Subject: Application for Graduate Trainee Position
📎 Attachments: Felix Lau Pangestu_CV.pdf, cover_letter_Cathay.pdf, Transcript_HKBU.pdf, Recommendation_Letters.pdf

📄 TO USE THIS TOOLKIT:
1. Modify the variables in Section 3
2. Run all cells
3. Uncomment the send_email line above to send
4. Install docx2pd